# Initial Setup or Reset

## Warning!!!!
### Running this code will delete an existing database as specified in the state dictionary below.

We need a way to save state throughout the project.  We will initially login as the ACCOUNTADMIN role in order to setup some additional users as well as the compute resources we will need. 

We will specify a couple of different compute resources which allows us to scale up and down easily.  Most of the workflow can use an extra-small warehouse but for certain tasks (ie. feature engineering and model training) we may need larger compute.  By specifying them in the state dictionary we can easily select the correct compute for any particular task.
  
Update the \<USERNAME>, \<ACCOUNTNAME>, \<DOMAIN> in the state dictionary below with the initial user that was created with your trial account.

Note: If you are running the US West (Oregon) region, you don't need to add the \<DOMAIN>.

In [1]:
state_dict = { 
  "account"   : "neb77596.us-east-1",
  "user"      : "jack",
  "password"  : "Yukti@24",
  "role"      : "ACCOUNTADMIN",
  "warehouse" : "A5_WH",
  "database"  : "A5_DB",
  "schema"    : "A5_SCHEMA"
                            
}

In [2]:
import json
with open('./connection.json', 'w') as sdf:
    json.dump(state_dict, sdf)

We will connect with username and password.  In a non-demo system it is very important to use properly secured passwords with secret managers and/or oauth.

In [5]:
# !pip install snowpark


In [3]:
import snowflake.snowpark as snp
import json
import getpass

account_admin_password = getpass.getpass('Enter password for user with ACCOUNTADMIN role access')

with open('./connection.json') as sdf:
    state_dict = json.load(sdf)    
state_dict['password'] = account_admin_password

session = snp.Session.builder.configs(state_dict).create()

ModuleNotFoundError: No module named 'snowflake'

We will also use a specific AWS S3 role for accessing pre-staged files to speed up the hands-on-lab.

In [6]:
state_dict['connection_parameters']['download_base_url'] = 's3://sfquickstarts/Summit 2022 Keynote Demo/click_data/'

To run this without access to pre-staged files run the following cell instead of the cell above.

In [5]:
#state_dict['connection_parameters']['download_base_url'] = 'https://s3.amazonaws.com/tripdata/'

Create a sample user which will be used for the hands-on-lab.  Normally you will have different roles (and possibly different users) for data scientists, data engineers, ML engineers, etc.

In [11]:
session.use_role('securityadmin')

demo_username='jack'
project_role='ACCOUNTADMIN'

session.sql("CREATE USER IF NOT EXISTS "+demo_username+\
            " LOGIN_NAME = '"+demo_username+"'"+\
            " FIRST_NAME = 'SNOWPARK'"+\
            " LAST_NAME = 'HOL'"+\
            " EMAIL = 'jack@hol.snowpark'"+\
            " DEFAULT_ROLE = '"+project_role+"'"+\
            " MUST_CHANGE_PASSWORD = FALSE")\
        .collect()

session.sql("GRANT ROLE "+project_role+" TO USER "+demo_username).collect()

session.use_role('sysadmin')
session.sql("GRANT CREATE DATABASE ON ACCOUNT TO ROLE "+project_role).collect()

[Row(status='Statement executed successfully.')]

In [12]:
session.use_role('securityadmin')
demo_user_password=getpass.getpass('Enter a new password for the demo user '+demo_username)
session.sql("ALTER USER "+demo_username+" SET PASSWORD = '"+demo_user_password+"'").collect()

Enter a new password for the demo user jack ········


Failed to execute query [queryID: 01a8aac9-0404-3370-0035-798700031042] ALTER USER jack SET PASSWORD = 'Yukti@24'
003002 (28P01): SQL execution error:
New password rejected by current password policy. Reason: 'PRIOR_USE'


SnowparkSQLException: (1304): 003002 (28P01): SQL execution error:
New password rejected by current password policy. Reason: 'PRIOR_USE'

Create compute instances as specified in the state dictionary.

In [13]:
# session.use_role('ACCOUNTADMIN')
# project_role='ACCOUNTADMIN'

# for wh in state_dict['compute_parameters'].values():
#     if wh != "train_warehouse":
#         session.sql("CREATE WAREHOUSE IF NOT EXISTS "+wh+\
#                 " WITH WAREHOUSE_SIZE = '"+wh.split('_')[0]+\
#                 "' WAREHOUSE_TYPE = 'STANDARD' AUTO_SUSPEND = 60 AUTO_RESUME = TRUE initially_suspended = true;")\
#             .collect()
#     elif wh == "train_warehouse":
#         session.sql("CREATE WAREHOUSE IF NOT EXISTS "+wh+\
#                 " WITH WAREHOUSE_SIZE = '"+wh.split('_')[0]+\
#                 "' WAREHOUSE_TYPE = 'HIGH_MEMORY' MAX_CONCURRENCY_LEVEL = 1 AUTO_SUSPEND = 60 AUTO_RESUME = TRUE initially_suspended = true;")\
#             .collect()
#     session.sql("GRANT USAGE ON WAREHOUSE "+wh+" TO ROLE "+project_role).collect() 
#     session.sql("GRANT OPERATE ON WAREHOUSE "+wh+" TO ROLE "+project_role).collect() 
    
# session.use_role(state_dict['connection_parameters']['role'])

Allow users to import data shares.

In [14]:
# session.use_role('ACCOUNTADMIN')
# # session.sql("GRANT IMPORT SHARE ON ACCOUNT TO "+project_role).collect()

[Row(status='Statement executed successfully.')]

Now update the state dictionary to use the non-admin account.

In [17]:
# state_dict['connection_parameters']['user'] = demo_username
# state_dict['connection_parameters']['password'] = demo_user_password
# state_dict['connection_parameters']['role'] = project_role
# state_dict['connection_parameters']['database'] = 'Assignment4_'+demo_username
# state_dict['connection_parameters']['schema'] = 'DEMO'

Save the updated state dictionary for project team use.

In [19]:
# import json
# with open('./connection.json', 'w') as sdf:
#     json.dump(state_dict, sdf)

Create a python function to simplify the users' steps of starting a session.

In [21]:
# %%writefile dags/snowpark_connection.py
# def snowpark_connect(state_file='./connection.json'):
#     import snowflake.snowpark as snp
#     import json
    
#     with open(state_file) as sdf:
#         state_dict = json.load(sdf)    
    
#     session=None
#     session = snp.Session.builder.configs(state_dict["connection_parameters"]).create()
#     session.use_warehouse(state_dict['compute_parameters']['default_warehouse'])
#     return session, state_dict

Writing dags/snowpark_connection.py


Test the function that users will use.

In [22]:
# from dags.snowpark_connection import snowpark_connect
# session, state_dict = snowpark_connect()

Make sure the user has access to each compute instance.

In [23]:
# session.get_current_warehouse()

'"XSMALL_WH"'

In [24]:
# for wh in state_dict['compute_parameters'].keys():
#     session.use_warehouse(state_dict['compute_parameters'][wh])

In [25]:
# session.get_current_warehouse()

'"XXLARGE_SNOWPARKOPT_WH"'

Create the database and schema for this project.

In [26]:
session.close()